In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +120 -delete -print


Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [8]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False, key=abs)


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR', 'MRVL', 'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

In [6]:
def run_command_with_check(command):
  """Runs a command, prints stderr, and raises an exception if it fails."""
  process = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

  if process.returncode != 0:
    print("stderr:", process.stderr)  # Print stderr only on error
    raise subprocess.CalledProcessError(process.returncode, process.args, output=process.stdout, stderr=process.stderr)
  else:
    # Print or process stdout if needed
    #print("stdout:", process.stdout)
    pass

  return process.returncode


In [5]:
import os
import subprocess
def run_model_nn(prefix, version, do_train=False):
    # copy drive/MyDrive/colab/results/nnreturn.py to local folder
    tickers = f"{prefix}tickers.csv"
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/results/nnreturn.py .")
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/results/{tickers} .")
    if not do_train:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/results/{prefix}_{version}_*.pth .")
      # Run nnreturn.py in results folder and throw exception if error
      today = pd.to_datetime('today').strftime('%m%d')
      run_command_with_check(f"rename 's/{version}/{today}/' *.pth")
      run_command_with_check(f"python3 nnreturn.py --tickers-file {prefix}tickers.csv --mode analyze")
      run_command_with_check(f"cp {prefix}*predictions.csv results")
      # Get today's date

      return show_prediction(prefix, today)
    else:
      raise Exception("Not implemented")


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

In [ ]:

df = show_prediction('etf', '0319').head(10)
df

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
31,URA,24.45,2025-03-18,2025-04-15,24.61,26.26,26.06,0.048807
4,DAPP,10.08,2025-03-18,2025-04-15,9.52,10.89,11.02,0.039352
44,XLP,79.75,2025-03-18,2025-04-15,79.18,84.27,82.91,0.029718
24,MTUM,200.36,2025-03-18,2025-04-15,201.70,204.22,211.03,0.026402
38,XLB,86.91,2025-03-18,2025-04-15,85.89,91.50,90.03,0.025659
14,IXC,40.93,2025-03-18,2025-04-15,40.44,43.18,42.27,0.025246
27,SMH,225.42,2025-03-18,2025-04-15,223.54,237.30,231.29,0.023467
49,XME,58.35,2025-03-18,2025-04-15,58.63,59.82,60.70,0.023422
13,IPO,38.69,2025-03-18,2025-04-15,38.37,39.68,40.34,0.019988
20,JETS,21.65,2025-03-18,2025-04-15,21.07,23.45,21.65,0.018784


In [ ]:
run_model_nn('etf', '0202')

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
25,ONLN,39.53,2025-04-17,2025-05-15,38.56,43.89,41.66,0.046547
31,URA,22.87,2025-04-17,2025-05-15,22.95,24.86,23.32,0.036729
39,XLC,90.21,2025-04-17,2025-05-15,88.96,94.82,95.64,0.032480
48,XLY,185.40,2025-04-17,2025-05-15,182.42,196.79,194.70,0.031841
0,ARKF,32.56,2025-04-17,2025-05-15,32.04,35.15,33.46,0.030405
9,FDN,211.93,2025-04-17,2025-05-15,205.40,231.13,218.53,0.030309
5,DIA,391.36,2025-04-17,2025-05-15,382.35,418.51,407.83,0.029478
50,XNTK,177.86,2025-04-17,2025-05-15,174.22,194.06,180.70,0.028862
20,JETS,18.88,2025-04-17,2025-05-15,18.87,20.09,19.29,0.028425
35,VUG,352.14,2025-04-17,2025-05-15,342.50,378.89,364.62,0.028010


In [ ]:
run_model_nn('pennystock', '1228').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
15,CHGG,0.60,2025-04-23,2025-05-21,0.57,1.16,1.07,0.555556
57,REI,0.88,2025-04-23,2025-05-21,0.79,1.33,1.45,0.352273
58,RIG,2.17,2025-04-23,2025-05-21,1.95,3.56,3.04,0.313364
20,DM,4.96,2025-04-01,2025-04-29,4.80,8.05,6.50,0.300403
70,VTEX,5.16,2025-04-23,2025-05-21,4.70,5.80,9.59,0.297804
55,PTON,6.09,2025-04-23,2025-05-21,3.20,5.99,3.97,-0.279693
63,SLE,0.28,2025-04-23,2025-05-21,0.26,0.35,0.46,0.273810
65,SST,0.31,2025-04-23,2025-05-21,0.30,0.53,0.35,0.268817
13,BLNK,0.72,2025-04-23,2025-05-21,0.45,1.30,0.95,0.250000
4,ACHR,8.16,2025-04-23,2025-05-21,4.23,8.39,5.92,-0.242647


In [ ]:
run_model_nn('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
19,FDX,215.88,2025-04-23,2025-05-21,212.46,242.33,233.78,0.063199
34,QCOM,142.90,2025-04-23,2025-05-21,140.66,158.85,153.34,0.056333
1,ACN,294.56,2025-04-23,2025-05-21,291.41,320.95,309.08,0.042730
2,ADBE,357.65,2025-04-23,2025-05-21,355.82,380.70,379.86,0.040477
40,UPS,99.00,2025-04-23,2025-05-21,95.83,104.98,106.85,0.035892
15,DELL,90.61,2025-04-23,2025-05-21,90.18,99.42,91.91,0.035610
33,PYPL,63.65,2025-04-23,2025-05-21,59.74,68.96,68.93,0.034983
26,META,529.75,2025-04-23,2025-05-21,503.05,581.74,556.46,0.032720
30,NVDA,103.62,2025-04-23,2025-05-21,98.30,112.89,109.60,0.031944
24,KO,73.43,2025-04-23,2025-05-21,72.71,79.01,74.46,0.026737


In [9]:
run_model_nn('country', '0224').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,ECH,31.67,2025-04-25,2025-05-23,29.52,32.54,29.63,-0.034944
7,EWI,44.04,2025-04-25,2025-05-23,40.22,45.26,42.10,-0.034363
11,EWP,40.86,2025-04-25,2025-05-23,36.79,42.01,41.18,-0.021211
17,ILF,24.90,2025-04-25,2025-05-23,23.96,25.83,23.37,-0.020616
21,THD,53.11,2025-04-25,2025-05-23,52.15,56.15,54.28,0.020398
6,EWG,39.36,2025-04-25,2025-05-23,37.23,40.48,38.05,-0.019648
4,EPHE,26.26,2025-04-25,2025-05-23,26.09,27.18,26.96,0.018406
9,EWC,42.04,2025-04-25,2025-05-23,41.29,43.43,43.39,0.015779
20,KWT,35.71,2025-04-25,2025-05-23,34.17,36.68,34.69,-0.014842
2,FLN,18.61,2025-04-25,2025-05-23,17.94,19.28,17.79,-0.014687


In [ ]:
run_model_nn('global', '0225').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,COTY,4.98,2025-04-25,2025-05-23,4.90,6.00,6.01,0.131861
60,THQQF,11.23,2025-04-24,2025-05-22,11.47,13.28,12.97,0.119620
27,GLOB,118.72,2025-04-25,2025-05-23,118.43,142.37,132.32,0.103774
13,DNNGY,13.05,2025-04-25,2025-05-23,12.81,14.01,14.77,0.062324
34,MGA,34.12,2025-04-25,2025-05-23,32.25,36.00,40.01,0.057640
1,ADDDF,248.70,2025-04-24,2025-05-22,244.89,271.26,269.23,0.052647
28,HCMLY,22.07,2025-04-25,2025-05-23,19.87,22.23,20.70,-0.051503
26,GLCNF,3.55,2025-04-25,2025-05-23,3.48,3.79,3.89,0.047887
35,RACE,458.45,2025-04-25,2025-05-23,450.75,498.41,488.70,0.045450
3,BAK,4.01,2025-04-25,2025-05-23,3.87,4.49,4.19,0.043225


In [10]:
run_model_nn('valuestock', '0301').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
0,AAL,9.75,2025-04-25,2025-05-23,9.71,11.30,10.93,0.091966
38,FIS,79.26,2025-04-25,2025-05-23,71.36,78.02,68.53,-0.083565
29,DAL,41.58,2025-04-25,2025-05-23,41.67,47.85,45.44,0.081930
3,AMR,126.20,2025-04-25,2025-05-23,123.92,145.58,139.42,0.080085
34,EMN,75.84,2025-04-25,2025-05-23,75.92,90.49,78.07,0.074543
77,SBLK,14.49,2025-04-25,2025-05-23,14.51,16.53,15.49,0.070393
9,BAH,119.59,2025-04-25,2025-05-23,118.64,135.64,129.03,0.068400
23,CG,39.03,2025-04-25,2025-05-23,38.82,43.33,42.88,0.067811
46,HPE,16.24,2025-04-25,2025-05-23,15.89,18.10,17.83,0.063629
74,RH,185.49,2025-04-25,2025-05-23,174.08,206.49,209.80,0.060920


In [ ]:
show_prediction('commodity', '0417').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
14,VTIP,49.86,2025-04-17,2025-05-15,48.67,53.96,49.66,0.018117
1,FXB,127.54,2025-04-17,2025-05-15,125.01,132.84,130.33,0.014531
2,FXE,105.03,2025-04-17,2025-05-15,102.86,108.27,107.18,0.010219
9,SOYB,21.66,2025-04-17,2025-05-15,21.17,22.45,21.82,0.007079
3,FXF,108.41,2025-04-17,2025-05-15,106.22,111.30,109.74,0.006242


In [ ]:
show_prediction('crypto', '0417').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
7,FIL-USD,2.38,2025-04-17,2025-05-15,2.31,2.94,3.21,0.184874
13,LINK-USD,12.43,2025-04-17,2025-05-15,12.07,17.39,13.92,0.163315
11,ICP-USD,4.69,2025-04-17,2025-05-15,4.31,5.77,5.57,0.112296
6,DOGE-USD,0.16,2025-04-17,2025-05-15,0.15,0.20,0.18,0.104167
18,ATOM-USD,4.17,2025-04-17,2025-05-15,3.78,4.77,4.97,0.080735


In [ ]:
show_prediction('techstock', '0417').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
9,BYRN,21.63,2025-04-17,2025-05-15,21.14,25.50,24.82,0.101248
1,ACLS,44.69,2025-04-17,2025-05-15,43.51,50.78,51.62,0.088312
79,UPWK,12.74,2025-04-17,2025-05-15,11.77,14.82,14.55,0.076400
24,ENPH,52.30,2025-04-17,2025-05-15,51.65,58.45,58.33,0.073486
26,ETSY,44.20,2025-04-17,2025-05-15,42.42,52.49,45.89,0.061840
14,CVNA,210.39,2025-04-17,2025-05-15,198.03,238.39,232.69,0.060111
58,PD,14.71,2025-04-17,2025-05-15,14.03,16.45,16.25,0.058917
74,TTD,50.03,2025-04-17,2025-05-15,46.01,58.56,53.98,0.056366
4,AMBA,42.71,2025-04-17,2025-05-15,39.91,50.20,44.83,0.053149
0,ABNB,113.04,2025-04-17,2025-05-15,111.46,131.64,113.71,0.052164


In [ ]:
show_prediction('stock', '0328').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
17,INSG,8.66,2025-03-28,2025-04-25,8.39,10.68,10.86,0.152040
139,INDI,2.05,2025-03-28,2025-04-25,2.01,2.55,2.42,0.134959
174,ASPN,6.64,2025-03-28,2025-04-25,6.36,8.10,7.74,0.114458
207,ZETA,13.57,2025-03-28,2025-04-25,12.94,16.09,16.25,0.112257
149,APPF,219.08,2025-03-28,2025-04-25,213.65,275.89,235.69,0.103448
26,ENPH,62.37,2025-03-28,2025-04-25,61.03,75.49,69.79,0.102613
136,LAZR,5.81,2025-03-28,2025-04-25,5.47,6.73,6.98,0.100402
39,IOT,38.75,2025-03-28,2025-04-25,37.50,46.62,43.18,0.095054
21,CREX,1.95,2025-03-28,2025-04-25,1.80,2.40,2.19,0.092308
65,TIXT,2.68,2025-03-28,2025-04-25,2.60,3.33,2.83,0.089552


In [ ]:
show_prediction('buystocks', '0402').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
113,TWLO,100.56,2025-04-02,2026-03-18,100.08,198.04,196.52,0.639618
1,SE,136.18,2025-04-02,2026-03-18,126.26,274.83,265.06,0.630562
102,CHGG,0.62,2025-04-02,2026-03-18,0.62,1.23,1.18,0.629032
160,JD,41.14,2025-04-02,2026-03-18,40.49,82.17,78.32,0.628423
361,ZIM,15.90,2025-04-02,2026-03-18,15.38,32.40,29.40,0.618029
211,RDVT,38.94,2025-04-02,2026-03-18,34.40,78.57,75.95,0.617189
298,AMBA,51.93,2025-04-02,2026-03-18,52.18,99.94,97.64,0.603184
27,INSG,8.55,2025-04-02,2026-03-18,7.33,17.34,16.42,0.601949
194,TSLA,281.00,2025-04-02,2026-03-18,244.83,566.54,533.60,0.595457
319,IDCC,209.16,2025-04-02,2026-03-18,199.35,402.97,397.00,0.592593


In [11]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +120 -delete -print

sending incremental file list
country_0426_predictions.csv
valuestock_0426_predictions.csv

sent 6,748 bytes  received 54 bytes  13,604.00 bytes/sec
total size is 6,535  speedup is 0.96
